In [1]:
import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn import neighbors

In [2]:
df = pd.read_csv("final.csv")
display(df)
print(df.columns)

,isbn,title,author,publication,publisher,desc,format,pages,rating,rating_count,...,nonfiction,horror,thriller,fantasy,mystery,history,romance,classic,crime,comedy
0,0399135782,The Kitchen God's Wife,Amy Tan,1991.0,Putnam Pub Group,Winnie and Helen have kept each other's worst ...,Paperback,416,3.99,66434.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0440234743,The Testament,John Grisham,1999.0,Dell,"In a plush Virginia office, a rich, angry old ...",Paperback,535,3.84,92234.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0609804618,Our Dumb Century: The Onion Presents 100 Years...,The Onion,1999.0,Three Rivers Press,The Onion has quickly become the world's most ...,Paperback,164,4.17,12031.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,0345402871,Airframe,Michael Crichton,1997.0,Ballantine Books,The twin jet plane en route to Denver from Hon...,Paperback,431,3.69,69392.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0345417623,Timeline,Michael Crichton,2000.0,Ballantine Books,"In an Arizona desert, a man wanders in a daze,...",Mass Market Paperback,489,3.84,170382.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9314,0590974092,"Sarah, Plain and Tall",Patricia Maclachlan,1996.0,Scholastic,This beloved Newbery Medal–winning book is the...,Paperback,64,3.73,63238.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
9315,2859406735,Armadale,Wilkie Collins,2000.0,Phã?Â©Bus,The novel has a convoluted plot about two dist...,Paperback,721,3.89,5307.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
9316,0441702902,Radiance,Anne Maybury,1980.0,Ace Books,Riley has crossed the bridge into the afterlif...,Paperback,183,3.57,14263.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9317,0345298608,The Broken Sword,Northam Anderson,1981.0,Del Rey Books,Thor broke the sword Tyrfing to save the roots...,Paperback,274,3.86,4465.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


Index(['isbn', 'title', 'author', 'publication', 'publisher', 'desc', 'format',
       'pages', 'rating', 'rating_count', 'review_count', 'genres', 'img_url',
       'fiction', 'nonfiction', 'horror', 'thriller', 'fantasy', 'mystery',
       'history', 'romance', 'classic', 'crime', 'comedy'],
      dtype='object')


In [3]:
df2 = df.copy()

# By author

In [68]:
'''
Returns a list of other books by the author
    returns a numpy array
'''

def recommend_by_author(_df, title):
    entry = _df[_df['title'] == title]
    author = entry['author'].to_numpy()
    display(author)
    
#     data = _df[_df['title'] != title]
    
    books = _df[(_df['author'].to_numpy() == author) &
                (_df['rating'] >= 3) &
                (_df['title'] != title)]
    out = books.to_numpy()

    return (out)


In [69]:
title = "The Tao of Pooh"
au_recs = recommend_by_author(df, title)

array(['Benjamin Hoff'], dtype=object)

In [73]:
# 'Benjamin Hoff'
x = df[df['author'] == 'Benjamin Hoff']
display(x)

,isbn,title,author,publication,publisher,desc,format,pages,rating,rating_count,...,nonfiction,horror,thriller,fantasy,mystery,history,romance,classic,crime,comedy
18,0140067477,The Tao of Pooh,Benjamin Hoff,1983.0,Penguin Books,The Wisdom of Pooh.Is there such thing as a We...,Paperback,176,4.02,85742.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
# title = "The Kitchen God's Wife"
# au_recs = recommend_by_author(df, title)

In [65]:
print(au_recs)

[]


# kNN Implementation

In [7]:
df2.loc[ (df2['rating'] >= 0) & (df2['rating'] <= 1), 'rating_between'] = "0 to 1"
df2.loc[ (df2['rating'] > 1) & (df2['rating'] <= 2), 'rating_between'] = "1 to 2"
df2.loc[ (df2['rating'] > 2) & (df2['rating'] <= 3), 'rating_between'] = "2 to 3"
df2.loc[ (df2['rating'] > 3) & (df2['rating'] <= 4), 'rating_between'] = "3 to 4"
df2.loc[ (df2['rating'] > 4) & (df2['rating'] <= 5), 'rating_between'] = "4 to 5"

In [8]:
rating_df = pd.get_dummies(df2['rating_between'])

In [9]:
g_types = ['fiction', 'nonfiction', 'horror', 'thriller', 'fantasy', 'mystery', 'history', 'romance', 'classic', 'crime', 'comedy']
genres = df2[g_types]

features = pd.concat([rating_df, df2['rating'], df2['rating_count'], df2['pages'], df2[g_types]], axis=1)

print(features)

      1 to 2  2 to 3  3 to 4  4 to 5  rating  rating_count  pages  fiction  \
0          0       0       1       0    3.99       66434.0    416      1.0   
1          0       0       1       0    3.84       92234.0    535      1.0   
2          0       0       0       1    4.17       12031.0    164      1.0   
3          0       0       1       0    3.69       69392.0    431      1.0   
4          0       0       1       0    3.84      170382.0    489      1.0   
...      ...     ...     ...     ...     ...           ...    ...      ...   
9314       0       0       1       0    3.73       63238.0     64      1.0   
9315       0       0       1       0    3.89        5307.0    721      1.0   
9316       0       0       1       0    3.57       14263.0    183      0.0   
9317       0       0       1       0    3.86        4465.0    274      1.0   
9318       0       0       1       0    3.98       20260.0    657      1.0   

      nonfiction  horror  thriller  fantasy  mystery  history  

In [10]:
min_max_scaler = MinMaxScaler()
features = min_max_scaler.fit_transform(features)

model = neighbors.NearestNeighbors(n_neighbors=10, algorithm='auto')
model.fit(features)
dist, idlist = model.kneighbors(features)

In [11]:
def kNN_recommender(_df, idlist, title):
    rec_titles = []

    idx = df2[df2['title'] == title].index
    bookID = idx[0]
    for newid in idlist[bookID]:
        if (df2.loc[newid].title == title):
            continue
        else:
            rec_titles.append(df2.loc[newid].title)
    
    books = _df[(_df['rating'] >= 3) &
                (_df['title'].isin(rec_titles))]
    stuff = ['isbn', 'title', 'author', 'rating', 'pages', 'desc', 'img_url']
    print(books['title'])
    out = books.to_numpy()
    
    return out

In [12]:
title = "The Tao of Pooh"
result = kNN_recommender(df, idlist, title)
# print(r)

1784                       Feel the Fear and Do It Anyway
2730                When Bad Things Happen to Good People
5701    The Art of Fiction: A Guide for Writers and Re...
6024                            The Conquest of Happiness
7327                                              On Fire
7365                       Tractatus Logico-Philosophicus
7731                         The Society of the Spectacle
8074          Spilling Open: The Art of Becoming Yourself
8233                     Seven Cats and the Art of Living
Name: title, dtype: object


In [13]:
NUM_REC = 5

def getRecommendations(_df, readBooks):
    books = list()
    if (len(readBooks) == 0):
        randoms = _df.sample(n = NUM_REC)
        out = randoms['isbn'].to_numpy()
    return out

In [23]:
isbns = getRecommendations(df, [])

# print(type)
x = df[df['isbn'].isin(isbns)]
print(x['isbn'])
for i in x['isbn']:
    print(i)

2029    0393049701
5677    039457589X
6957    0374453098
7186    0553111132
7892    158081185X
Name: isbn, dtype: object
0393049701
039457589X
0374453098
0553111132
158081185X


In [74]:
print(df.title)

0                                  The Kitchen God's Wife
1                                           The Testament
2       Our Dumb Century: The Onion Presents 100 Years...
3                                                Airframe
4                                                Timeline
                              ...                        
9314                                Sarah, Plain and Tall
9315                                             Armadale
9316                                             Radiance
9317                                     The Broken Sword
9318                                           Pathfinder
Name: title, Length: 9319, dtype: object
